# Desafio Final - Módulo 2 - Modelo Preditivo para Derrame Cerebral

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Desafio Final - Módulo 2 - Modelo Preditivo para Derrame Cerebral") \
        .getOrCreate()
    
spark.version

'3.2.1'

In [2]:
# Lendo o arquivo com os dados dos pacientes
patient_df = spark.read.csv("/home/jovyan/work/stroke_data.csv", header=True, inferSchema=True)
patient_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [3]:
patient_df.show(10)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

In [4]:
# Retorna os registros nulos para os campos idedntificados
from pyspark.sql.functions import col

dataNull = patient_df.filter(col("gender").isNull() | col("age").isNull() | col("hypertension").isNull() | col("heart_disease").isNull() | col("ever_married").isNull() | col("work_type").isNull() | col("Residence_type").isNull() | col("avg_glucose_level").isNull() | col("bmi").isNull() | col("smoking_status").isNull() | col("stroke").isNull())

dataNull.show(100)

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
|  0|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|bmi|smoking_status|stroke|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+



## Pergunta 1 - Quantos registros existem no arquivo

In [5]:
print(f"Total de registros: {patient_df.count()}")

Total de registros: 67135


## Pergunta 2 - Quantas colunas existem no arquivo? Quantas são númericas?

In [6]:
# Quantas colunas existem no arquivo
print(f"Total de colunas: {len(patient_df.columns)}")
# Quantas colunas são númericas
print(f"Total de colunas númericas: {len([coluna[1] for coluna in patient_df.dtypes if coluna[1] in ['int', 'double']])}")

Total de colunas: 12
Total de colunas númericas: 7


## Pergunta 3 - Quantos pacientes sofream e não sofreram derrame (stroke)
##### 1 para quem sofreu derrame (stroke) e 0 para quem não sofreu

In [7]:
patient_df.groupBy("stroke").count().show()

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



## Pergunta 4 - Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças

In [8]:
from pyspark.sql import *

patient_df.createOrReplaceTempView('table')

In [9]:
spark.sql('SELECT work_type, count(work_type) FROM table WHERE stroke = 1 GROUP BY work_type ORDER BY count(*) DESC').show()

+-------------+----------------+
|    work_type|count(work_type)|
+-------------+----------------+
|      Private|           23711|
|Self-employed|           10807|
|     Govt_job|            5164|
|     children|             520|
| Never_worked|              85|
+-------------+----------------+



## Pergunta 5 - Escreva uma consulta com spark.sql para determinar a proporção, por genêro, de participantes do estudo

In [10]:
spark.sql('SELECT gender, count(gender) FROM table GROUP BY gender').show()

+------+-------------+
|gender|count(gender)|
+------+-------------+
|Female|        39530|
| Other|           11|
|  Male|        27594|
+------+-------------+



## Pergunta 6 - Quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos

In [11]:
# Hipertensos
data_hypertension = spark.sql('SELECT stroke, count(stroke) as qtd FROM table WHERE hypertension = 1 GROUP BY stroke')
stroke = data_hypertension.select('qtd').where('stroke = 1').collect()[0][0]
no_stroke = data_hypertension.select('qtd').where('stroke = 0').collect()[0][0]

prob_stroke_hypertension = stroke / (stroke + no_stroke) * 100
print(f"A probabilidade de hipertensos ter derrame foi de {prob_stroke_hypertension:.2f}%")

# Não Hipertensos
data_no_hypertension = spark.sql('SELECT stroke, count(stroke) as qtd FROM table WHERE hypertension = 0 GROUP BY stroke')
stroke = data_no_hypertension.select('qtd').where('stroke = 1').collect()[0][0]
no_stroke = data_no_hypertension.select('qtd').where('stroke = 0').collect()[0][0]

prob_stroke_no_hypertension = stroke / (stroke + no_stroke) * 100
print(f"A probabilidade de não hipertensos ter derrame foi de {prob_stroke_no_hypertension:.2f}%")

A probabilidade de hipertensos ter derrame foi de 80.03%
A probabilidade de não hipertensos ter derrame foi de 56.08%


## Pergunta 7 - Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?

In [12]:
spark.sql('SELECT age, count(age) FROM table WHERE stroke = 1 GROUP BY age ORDER BY COUNT(age) DESC').show(1)

+----+----------+
| age|count(age)|
+----+----------+
|79.0|      2916|
+----+----------+
only showing top 1 row



## Pergunta 8 - Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos

In [13]:
patient_df.where('stroke = 1').where('age > 50').count()

28938

## Pergunta 9 - Qual o mnível médio de glicose para pessoas que, respectivamente, sofrerame não sofreram derrame

In [14]:
spark.sql('SELECT stroke, AVG(avg_glucose_level) as glucose_avg FROM table GROUP BY stroke').show()

+------+------------------+
|stroke|       glucose_avg|
+------+------------------+
|     1|119.95307046938272|
|     0|103.60273130214506|
+------+------------------+



## Qual é o BMI (IMC) médio de quem sofreu e não sofreu derrame

In [15]:
patient_df.groupBy('stroke').avg('bmi').show()

+------+------------------+
|stroke|          avg(bmi)|
+------+------------------+
|     1|29.942490629729495|
|     0|27.989678933253657|
+------+------------------+



# Modelo de Árvorde de Decisão

In [16]:
patient_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [17]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
# Colunas númericas
numericCols = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]

# Todas as colunas
allCols = numericCols

# VectorAsssembler é um transformer
# Transforma um dataframe com colunas e um vetor com colunas
vecAssembler = VectorAssembler(inputCols=allCols, outputCol="features")

In [19]:
# Instanciar o modelo de aprendizado de máquina Árvore de Decisão
dtc = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')

In [20]:
# Um pipeline é uma sequencia de estágios
# É um estimador
pipeline = Pipeline(stages=[vecAssembler, dtc])

# Separa os dados em dados de treinamento e teste
train_data,test_data = patient_df.randomSplit([0.7,0.3])

# Pipeline é um estimador que recebe um dataframe e produz um Model
pipelineModel = pipeline.fit(train_data)

# Aplica o modelo do pipeline aos dados de teste
predictionsDF = pipelineModel.transform(test_data)

## Pergunta 11 - Qual a acurácia de um modelo construído

In [21]:
# acurácia: % de previsões corretas
evaluator = MulticlassClassificationEvaluator(metricName="accuracy", labelCol='stroke')
print(f"Acurácia: {evaluator.evaluate(predictionsDF) * 100:.4f}%")

Acurácia: 69.4576%


## Pergunta 12 - Adicione gênero e status de fumante. Verifique a acurácia do modelo

In [22]:
# Identifica as colunas categorias
numericCols = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]
categoricalCols = ["gender", "smoking_status"]

# Cria estimadores (que implementam fit()) que retornam funções que vão ser aplicadas para transformar o dataset
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=[x + "Index" for x in categoricalCols])
# Divide cada dado da categoria em uma coluna e atribui 1 quando for sua ocorrência para o registro
oneHotEncoder = OneHotEncoder(inputCols=stringIndexer.getOutputCols(), outputCols=[x + "OHE" for x in categoricalCols])

In [23]:
# Todas as colunas
allCols = [c + "OHE" for c in categoricalCols] + numericCols

# VectorAsssembler é um transformer
# Transforma um dataframe com colunas e um vetor com colunas
vecAssembler = VectorAssembler(inputCols=allCols, outputCol="features")

In [24]:
# Instanciar o modelo de aprendizado de máquina Árvore de Decisão
dtc_category = DecisionTreeClassifier(labelCol='stroke',featuresCol='features')

In [25]:
# Um pipeline é uma sequencia de estágios
# É um estimador
pipeline_category = Pipeline(stages=[stringIndexer, oneHotEncoder, vecAssembler, dtc_category])

# Separa os dados em dados de treinamento e teste
train_data,test_data = patient_df.randomSplit([0.7,0.3])

# Pipeline é um estimador que recebe um dataframe e produz um Model
pipelineModel_category = pipeline_category.fit(train_data)

# Aplica o modelo do pipeline aos dados de teste
predictionsDF_category = pipelineModel_category.transform(test_data)

In [26]:
evaluator_category = MulticlassClassificationEvaluator(metricName="accuracy", predictionCol="prediction", labelCol='stroke')
print(f"Acurácia: {evaluator_category.evaluate(predictionsDF_category) * 100:.4f}%")

Acurácia: 83.4980%


## Pergunta 13 - Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu na questão 12

In [27]:
dtc_category_model = pipelineModel_category.stages[3]

dtc_category_model.depth

5

In [28]:
dtc_category_model.featureImportances

SparseVector(9, {2: 0.4829, 3: 0.3321, 4: 0.1722, 5: 0.0012, 6: 0.0046, 8: 0.0071})

In [29]:
vecAssembler.getInputCols()

['genderOHE',
 'smoking_statusOHE',
 'age',
 'bmi',
 'hypertension',
 'heart_disease',
 'avg_glucose_level']

In [30]:
# A importância de cada atributo no modelo
list(zip(vecAssembler.getInputCols(), dtc_category_model.featureImportances))

[('genderOHE', 0.0),
 ('smoking_statusOHE', 0.0),
 ('age', 0.48288159134498126),
 ('bmi', 0.3320852685003182),
 ('hypertension', 0.17217748163670465),
 ('heart_disease', 0.0011788325496875699),
 ('avg_glucose_level', 0.0046063362987090895)]

In [31]:
for coluna in zip(vecAssembler.getInputCols(), dtc_category_model.featureImportances):
    print(f"Coluna: {coluna[0]}  Importancia: {coluna[1]:.4f}")

Coluna: genderOHE  Importancia: 0.0000
Coluna: smoking_statusOHE  Importancia: 0.0000
Coluna: age  Importancia: 0.4829
Coluna: bmi  Importancia: 0.3321
Coluna: hypertension  Importancia: 0.1722
Coluna: heart_disease  Importancia: 0.0012
Coluna: avg_glucose_level  Importancia: 0.0046


## Pergunta 15 - Quantos nodos a árvore de decisão possui?

In [33]:
display(dtc_category_model)
dtc_category_model.numNodes

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_ff4d697f7556, depth=5, numNodes=21, numClasses=2, numFeatures=9

21